# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [2]:
# Import necessary libraries
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda:0"

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, Dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format, DataCollatorForCompletionOnlyLM
from tqdm import tqdm
import random 
import torch

print(torch.cuda.device_count())

# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps" if torch.backends.mps.is_available() else "cpu"
# )


# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    device_map=device,
)#.to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-haiku-fullfinetune"
finetune_tags = ["smol-course", "module_1"]

1


# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [3]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [4]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
# ds = load_dataset("openai/gsm8k", "main")
# ds = load_dataset("bigcode/the-stack-smol", split="train")

# ds = load_dataset("davanstrien/haiku_dpo", split="train")

# def format_ds_text_label(sample):
#     return {"text":sample["question"], "label":sample["chosen"]}
# ds = ds.map(format_ds_text_label)
print(ds)
print(ds["train"][0]["messages"])

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})
[{'content': 'Hi there', 'role': 'user'}, {'content': 'Hello! How can I help you today?', 'role': 'assistant'}, {'content': "I'm looking for a beach resort for my next vacation. Can you recommend some popular ones?", 'role': 'user'}, {'content': "Some popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.", 'role': 'assistant'}, {'content': 'That sounds great. Are there any resorts in the Caribbean that are good for families?', 'role': 'user'}, {'content': 'Yes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.', 'role': 'assistant'}, {'content': "Okay, I'll look i

In [5]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
# dataset = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations", split="train")
# dataset = load_dataset(path="prithivMLmods/Math-Solve", split="train[:25000]")

# def format_dataset(sample):
#     content = [
#         {"role": "system", "content": "You are helpful"},
#         {"role": "user", "content": sample["problem"]}, 
#         {"role": "assistant", "content": sample["solution"]}
#     ]
#     return {"messages":content}

# dataset = dataset.map(format_dataset)


# dataset = load_dataset(path="Qurtana/medical-o1-reasoning-SFT-orpo",  split="train")
# def format(sample):
#     content = [
#         {"role": "system", "content": "You are helpful assistant"},
#         {"role": "user", "content": sample["prompt"]}, 
#         {"role": "assistant", "content": sample["accepted"]}
#     ]
#     return {"messages":content}

# dataset=dataset.map(format)

# dataset_train = dataset.select(range(20000))
# dataset_test = dataset.select(range(dataset.shape[0]-2000, dataset.shape[0]))

# print(dataset)


# dataset = load_dataset(path="openai/gsm8k", name="main")

# def format(sample):
#     pass
#     content = [
#         {"role": "system", "content": "You are helpful assistant"},
#         {"role": "user", "content": sample["question"]}, 
#         {"role": "assistant", "content": sample["answer"]}
#     ]
#     return {"messages":content}

# dataset=dataset.map(format)

# dataset_train = dataset["train"]
# dataset_test = dataset["test"]

# print(dataset)


dataset = load_dataset(path="davanstrien/haiku_dpo", split="train")

# data = []
# for row in tqdm(dataset.iter(batch_size=1), total=len(dataset)):
#     question = row["question"][0]
#     generations = row["generations"][0]
#     if len(question) > 2:
#         data.extend((question, x) for x in generations)

# random.shuffle(data)

# ds = Dataset.from_dict(
#     {
#         "question": [d[0] for d in data],
#         "generations": [d[1] for d in data]
#     }
# )

# data = [(x,y) for data]

def format(sample):
    pass
    messages = [
        {"role": "system", "content": "You are helpful assistant"},
        {"role": "user", "content": sample["question"]}, 
        {"role": "assistant", "content": "\n\n".join(sample["generations"])} 
    ]
    return {"messages":messages}


dataset=dataset.map(format)

train_rartio = .95 
idx_train = list(range(int(dataset.shape[0]*train_rartio)))
idx_test = range(idx_train[-1]+1, dataset.shape[0])
dataset_train = dataset.select(idx_train)
dataset_test = dataset.select(idx_test)

# dataset_train = dataset.select(range(15000))
# dataset_test = dataset.select(range(dataset.shape[0]-1000, dataset.shape[0]))

print(dataset_train)

dataset_train[0]["messages"]

Dataset({
    features: ['question', 'generation_model', 'generation_prompt', 'generations', 'scores', 'chosen', 'chosen_score', 'rejected', 'rejected_score', 'tie', 'difference_in_score', 'system', 'messages'],
    num_rows: 3916
})


[{'content': 'You are helpful assistant', 'role': 'system'},
 {'content': 'Can you compose a haiku about the serenity of mountain peaks?',
  'role': 'user'},
 {'content': "Majestic peaks so tall,\nSilent whispers of time's grace,\nNature's cathedral.\n\nMountaintop so high,\nSilence wraps around in peace,\nNature's breath in sighs.\n\nMajestic peaks soar,\nSilent wisdom in their heights,\nNature's calming song.\n\nMajestic mountain,  \nSerenity in every peak,  \nNature's quiet song.\n\nMajestic peaks soar,\nSilent whispers of stone and snow,\nNature's cathedral.\n\nMajestic peak so high,\nIn silence, nature's beauty,\nEternal peace reigns.\n\nMajestic peaks soar,\nSilence speaks, soul at ease,\nNature's grand cathedral.\n\nMajestic peaks soar,\nSilent whispers of stone and sky,\nNature's grand cathedral.\n\nMajestic peaks so high,\nSilent sentinels of time,\nNature's cathedral.\n\nMajestic peaks ascend,\nSilent, stoic, standing tall,\nNature's grand cathedral.\n\nMajestic peaks so high

In [18]:
print(dataset_train[0]["messages"][-1]["content"])

Majestic peaks so tall,
Silent whispers of time's grace,
Nature's cathedral.

Mountaintop so high,
Silence wraps around in peace,
Nature's breath in sighs.

Majestic peaks soar,
Silent wisdom in their heights,
Nature's calming song.

Majestic mountain,  
Serenity in every peak,  
Nature's quiet song.

Majestic peaks soar,
Silent whispers of stone and snow,
Nature's cathedral.

Majestic peak so high,
In silence, nature's beauty,
Eternal peace reigns.

Majestic peaks soar,
Silence speaks, soul at ease,
Nature's grand cathedral.

Majestic peaks soar,
Silent whispers of stone and sky,
Nature's grand cathedral.

Majestic peaks so high,
Silent sentinels of time,
Nature's cathedral.

Majestic peaks ascend,
Silent, stoic, standing tall,
Nature's grand cathedral.

Majestic peaks so high,
Silent whispers of the sky,
Nature's crown of pride.

Majestic peaks rise,
Silent strength in snow and stone,
Nature's cathedral.

Majestic peaks rise,
Silent sentinels of time,
Nature's grand cathedral.

Majes

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [6]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    # Small learning rate to prevent catastrophic forgetting
    learning_rate=1e-5,
    # Linear learning rate decay over training
    lr_scheduler_type="cosine",
    # Maximum combined length of prompt + completion
    max_seq_length=1512,
    # # Maximum length for input prompts
    # max_prompt_length=512,
    # Batch size for training
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # Helps with training stability by accumulating gradients before updating
    gradient_accumulation_steps=4,
    # Memory-efficient optimizer for CUDA, falls back to adamw_torch for CPU/MPS
    # optim="paged_adamw_8bit" if device == "cuda" else "adamw_torch",
    # Number of training epochs
    num_train_epochs=4,
    # When to run evaluation
    evaluation_strategy="steps",
    # Evaluate every 20% of training
    eval_steps=0.2,
    # Log metrics every step
    logging_steps=1,
    # Gradual learning rate warmup
    warmup_steps=100,
    # Disable external logging
    report_to="wandb",
    # Where to save model/checkpoints
    output_dir=finetune_name,
    # Enable MPS (Metal Performance Shaders) if available
    # use_mps_device=device == "mps",
    hub_model_id=finetune_name,
    # Use bfloat16 precision for faster training
    bf16=True,
)



# def format_cols(sample):
#     return {"text":sample["question"], "label":sample["answer"]}

# ds = ds.map(format_cols)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    processing_class=tokenizer,
)


# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`



/home/matte/smol-course/.venv/lib/python3.10/site-packages/transformers/training_args.py:1576: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
# print(len(trainer.train_dataset[0]["input_ids"]))

# print(tokenizer.decode(trainer.train_dataset[0]["input_ids"]))

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [8]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


wandb: Currently logged in as: matteob-90-hotmail-it. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss
196,1.681000,1.572136
392,1.532200,1.468184
588,1.409500,1.431804
784,1.508700,1.419608


In [9]:
trainer.push_to_hub(tags=finetune_tags)

training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/matteobonotto/SmolLM2-FT-haiku-fullfinetune/commit/4106f047e5716be03aa22391be06f53f14c67c11', commit_message='End of training', commit_description='', oid='4106f047e5716be03aa22391be06f53f14c67c11', pr_url=None, repo_url=RepoUrl('https://huggingface.co/matteobonotto/SmolLM2-FT-haiku-fullfinetune', endpoint='https://huggingface.co', repo_type='model', repo_id='matteobonotto/SmolLM2-FT-haiku-fullfinetune'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [16]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about Italy"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.
out = model.generate(**inputs, max_new_tokens=100)

print(
    tokenizer.decode(out[0], skip_special_tokens=True)
)

user
Write a haiku about Italy

Italy's beauty,
A gentle breeze whispers,
Nature's symphony.

Italy's beauty,
A gentle breeze whispers,
Nature's symphony.

Italy's beauty,
A gentle breeze whispers,
Nature's symphony.

Italy's beauty,
A gentle breeze whispers,
Nature's symphony.

Italy's beauty,
A gentle breeze whispers,
Nature's symphony.

Italy's beauty,
A gentle breeze whispers,
Nature's symphony


In [14]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about python"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.
out = model.generate(**inputs, max_new_tokens=100)

print(
    tokenizer.decode(out[0], skip_special_tokens=True)
)

user
Write a haiku about python

Python's beauty,

A gentle, yet powerful force,

Nature's art, in code.

Python's beauty,

A gentle, yet powerful force,

Nature's art, in code.

Python's beauty,

A gentle, yet powerful force,

Nature's art, in code.

Python's beauty,

A gentle, yet powerful force,

Nature's art, in code.

Python's beauty


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.